# Training a Classifier on the *Salammbô* Dataset with PyTorch
Author: Pierre Nugues

We first need to import some modules

In [53]:
import torch
import torch.nn as nn
import random

In [54]:
random.seed(4321)
torch.manual_seed(4321)

### Reading the dataset
We can read the data from a file with the svmlight format or directly create numpy arrays

In [55]:
X = torch.tensor(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ])

y = torch.Tensor(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Scaling the Data
Scaling and normalizing are usually very significant with neural networks. We use sklean transformers. They consist of two main methods: `fit()` and `transform()`.

### Normalizing

In [56]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_norm = normalizer.fit_transform(X)
X_norm[:4]

array([[0.99807515, 0.06201605],
       [0.99789783, 0.06480679],
       [0.99787509, 0.06515607],
       [0.99793128, 0.06428964]])

### Standardizing

In [57]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X_norm)
X_scaled[:4]

array([[ 1.68336574, -1.7197772 ],
       [ 0.57376529, -0.56145427],
       [ 0.43143908, -0.41648279],
       [ 0.78308579, -0.77610221]])

## Creating PyTorch Tensors
PyTorch has its own implementation of matrices called tensors. They are more or less equivalent to NumPy arrays. We need to convert our dataset to these tensors and represent $\mathbf{y}$  as a column vector

In [58]:
Y = y.reshape((-1, 1))
Y[:4]

tensor([[0.],
        [0.],
        [0.],
        [0.]])

In [59]:
X_scaled = torch.Tensor(X_scaled)
X_scaled[:4]


tensor([[ 1.6834, -1.7198],
        [ 0.5738, -0.5615],
        [ 0.4314, -0.4165],
        [ 0.7831, -0.7761]])

## Creating a Model

In [60]:
input_dim = X_scaled.shape[1]

We define a classifier equivalent to a logistic regression

In [61]:
model1 = nn.Sequential(
    nn.Linear(input_dim, 1),
    nn.Sigmoid())

In [62]:
class Model1(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

And a model with one hidden layer

In [63]:
model2 = nn.Sequential(
    nn.Linear(input_dim, 4),
    nn.ReLU(),
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2, 1),
    nn.Sigmoid())

In [64]:
class Model2(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4)
        self.fc2 = nn.Linear(4, 2)
        self.fc3 = nn.Linear(2, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

We create the model. To try the network with one hidden layer, set `complex` to true

In [65]:
complex = True
seq = False

In [66]:
if seq:
    if complex:
        model = model2
    else:
        model = model1
else:
    if complex:
        model = Model2(input_dim)
    else:
        model = Model1(input_dim)

In [67]:
loss_fn = nn.BCELoss()    # binary cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Fitting the Model
We will show three methods: batch gradient descent, stochastic descent, and minibatches

### Batch Gradient Descent

We fit the whole dataset (batch gradient descent)

In [68]:
model.train()               # sets PyTorch in the train mode
for epoch in range(100):
    Y_pred = model(X_scaled)
    loss = loss_fn(Y_pred, Y)
    optimizer.zero_grad()   # resets the gradients
    loss.backward()         # gradient backpropagation
    optimizer.step()        # weight updates

### Stochastic Gradient Descent

or, we fit the model with a batch size of one item (stochastic gradient descent)

In [69]:
model.train()
for epoch in range(50):
    for x_scaled, y in zip(X_scaled, Y):
        y_pred = model(x_scaled)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Minibatch Gradient Descent

Or we fit it with mini-batches, first with a simple inner loop

In [70]:
batch_size = 4
model.train()
for epoch in range(50):
    # Would need to shuffle X and y
    for i in range(0, X_scaled.size()[0], batch_size):
        Y_batch_pred = model(X_scaled[i:i + batch_size])
        loss = loss_fn(Y_batch_pred, Y[i:i + batch_size])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Then with a dataloader

In [71]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_scaled, Y)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [72]:
model.train()
for epoch in range(50):
    for X_scaled_batch, Y_batch in dataloader:
        Y_batch_pred = model(X_scaled_batch)
        loss = loss_fn(Y_batch_pred, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## The weights

In [73]:
list(model.parameters())

[Parameter containing:
 tensor([[ 1.2483, -1.2677],
         [-0.3138, -0.3383],
         [-1.1183, -0.0452],
         [ 0.2574,  0.2446]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1402,  0.4605,  0.5205, -0.2965], requires_grad=True),
 Parameter containing:
 tensor([[-0.1809, -0.4764, -0.4410,  0.4405],
         [ 1.6513,  0.0579, -0.9737, -0.3093]], requires_grad=True),
 Parameter containing:
 tensor([0.1278, 0.5512], requires_grad=True),
 Parameter containing:
 tensor([[-0.4484, -1.9978]], requires_grad=True),
 Parameter containing:
 tensor([1.9692], requires_grad=True)]

Also in the form of a dictionary

In [74]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 1.2483, -1.2677],
                      [-0.3138, -0.3383],
                      [-1.1183, -0.0452],
                      [ 0.2574,  0.2446]])),
             ('fc1.bias', tensor([ 0.1402,  0.4605,  0.5205, -0.2965])),
             ('fc2.weight',
              tensor([[-0.1809, -0.4764, -0.4410,  0.4405],
                      [ 1.6513,  0.0579, -0.9737, -0.3093]])),
             ('fc2.bias', tensor([0.1278, 0.5512])),
             ('fc3.weight', tensor([[-0.4484, -1.9978]])),
             ('fc3.bias', tensor([1.9692]))])

## Prediction

### Probabilities

We compute the probabilities to belong to class 1 for all the training set

In [75]:
model.eval()
y_pred_proba = model(X_scaled)
y_pred_proba[:4]

tensor([[1.0381e-06],
        [1.2621e-02],
        [4.4951e-02],
        [2.1934e-03]], grad_fn=<SliceBackward0>)

### Classes

In [76]:
def predict_class(y_pred_proba):
    y_pred = torch.zeros(y_pred_proba.shape[0])
    for i in range(y_pred_proba.shape[0]):
        if y_pred_proba[i][0] >= 0.5:
            y_pred[i] = 1
    return y_pred

In [77]:
y_pred = predict_class(y_pred_proba)
y_pred

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Evaluation

In [78]:
from sklearn.metrics import classification_report

print(classification_report(Y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00        15

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



We computed the accuracy from the training set. This is not a good practice. We should use a dedicated test set instead.